In [1]:
# Import Library
import pandas as pd
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [2]:
#Load environment variables
load_dotenv(override=True)
MONGODB_URI = os.getenv("MONGODB_URI")

In [3]:
#initialize embeddings
embeddings=OllamaEmbeddings(model="mxbai-embed-large")

In [ ]:
#Model configuration with llama
model= OllamaLLM(model="llama3.2",
                temperature=0,
                convert_system_message_to_human=True)

In [5]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq_tb']
index_name = 'vector_index'

In [6]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index'
)

In [7]:
# PROMPT BARU UNTUK BPJS
BPJS_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Digital Resmi BPJS Kesehatan dan BPJS Ketenagakerjaan.
    Tugas Anda sebagai Customer Service adalah menjawab pertanyaan peserta berdasarkan data FAQ resmi yang telah disediakan.

    Tanggung jawab utama:
    - Memberikan jawaban yang akurat dan informatif sesuai dengan kategori FAQ BPJS Kesehatan dan BPJS Ketenagakerjaan.
    - Menyampaikan informasi dengan bahasa yang profesional, jelas, dan mudah dipahami oleh seluruh lapisan masyarakat.
    - Memberikan panduan teknis atau prosedur (jika diperlukan) dalam format langkah-langkah yang berurutan atau poin-poin.
    - Menanggapi pertanyaan atau peserta dengan solusi yang relevan dari konteks FAQ yang tersedia.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi yang diberikan.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan: "Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan."
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal atau informasi promosi di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi (dari FAQ):
    {context}

    Pertanyaan Peserta: {question}

    Jawaban Profesional:
    """
)

In [8]:
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5, "score_treshold":0.85 
                       }  # Mengambil 5 dokumen teratas

    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": BPJS_PROMPT}
)
def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})
        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi."
        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi."

faq_df = pd.read_csv('../data/final_dataset_clean.csv')
def answer_from_doc(question):
    row = faq_df[faq_df['question'].str.lower() == question.lower()]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

In [9]:
# TEST CASE 1: Pertanyaan di dalam konteks (formal dan informal)
print("\n\n UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset")
questions = [
    "Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?",
    "Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?",
    "Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?",
    "Apakah itu Promo?",
    "Apakah itu MotionPay?", 
    "Bagaimana jika lupa PIN EPS?",
    "Apakah sandi/PIN harus diubah secara berkala di kedua aplikasi?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
    "Apa itu BPJS Ketenagakerjaan?",
    "Apa saja program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerjaan?",
    "Apakah yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Buruh Harian Lepas) dapat mengikuti program JKP",
    "Bagaimana caranya badan usaha/pemberi kerja dapat menggunakan layanan Payment Reminder System(PRS)?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?",
    "Apakah aplikasi Electronic Payment System (EPS)?",
    "Apa perbedaan program Jaminan Hari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan?",
    "Bagaimana Mendaftarkan Bayi Baru Lahir?",
    "Apa itu GRC?",
    "Cara Membayar Iuran",
    "Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan?",
    "Kapan pembayaran iuran paling lambat?",
    "Bagaimana komitmen BPJS Kesehatan terkait gratifikasi?"
 
]

for q in questions:
    print(f"\n{'-'*5}")
    print(f"PERTANYAAN: {q}")
    chatbot_response = ask(q)
    print(f"JAWABAN CHATBOT:\n{chatbot_response}")
    print(f"\n{'-'*5}")

    
    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'='*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'='*50}")



 UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset

-----
PERTANYAAN: Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?
JAWABAN CHATBOT:
Pertanyaan peserta sangat penting untuk dijawab. Berikut adalah jawaban yang tepat:

Untuk menginstall aplikasi Jamsostek Mobile (JMO), silakan ikuti langkah-langkah berikut:


1.  Buka Playstore/ Appstore di perangkat Anda.

2.  Pencarian "Jamsostek Mobile" untuk menemukan aplikasi ini

3.  Klik aplikasi dan pilih "Instal"

4.  Jika terjadi masalah dengan instalasi, silakan hubungi Call Center BPJS Ketenagakerjaan atau kunjungi website https://sso.bpjsketenagakerjaan.go.id untuk mendapatkan bantuan lebih lanjut.

Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Aplikasi Jamsostek Mobile (JMO) dapat diunduh d

TOTAL SOAL : 26
(ada di dataset jawabannya + jawaban terjawab atau tidak)
- Pertanyaan -jawaban ada di dataset dan terjawab:13
- Pertanyaan -jawaban tidak ada di dataset dan terjawab:6
- Pertanyaan - jawaban ada di dataset dan tidak terjawab:5
- Pertanyaan  tidak ada di dataset dan tidak terjawab:2

In [10]:
# TEST CASE 2: Pertanyaan di luar konteks (untuk menguji penolakan)
# Pertanyaan ini untuk memastikan chatbot tidak mengarang jawaban.
print("\n\n UJI COBA 2: Pertanyaan di Luar Konteks ")
questions_out_of_context = [
    "Apa itu Allo Bank?",
    "Bagaimana cara mengajukan KPR?",
    "Siapa penemu listrik?",
    "Apa saja manfaat dari olahraga?",
    "Bagaimana cara membuat kue?",
    "Jelaskan apa itu Quantum Computing?",
    "Apa yang dimaksud dengan Machine Learning?",
    "Gimana cara buat foto HD dengan AI?",
    "Apa itu teknologi blockchain?",
    "Apa itu teknologi quantum?",
    "Apa kopi paling enak saat ini?",
    "Sebutkan Sepatu lari paling enak",
    "Kapan Indonesia Merdeka?",
    "Dimana kantor urusan agama terdekat",
    "Jelaskan tentang skema piala dunia?",
    "Siapa pencetak gol terbanyak piala dunia 2022?",
    "Jumlah total trofi Liverpool berapa?",
    "Hitung luas Kota Surabaya",
    "Dimana tempat syuting film laskar Pelangi?",
    "Dimana bom atom amerika saat perang dunia kedua dibuat?",
    "Bagaimana cara membedakan biji kopi?",
    "Jembatan merah ada Dimana?",
    "Siapa presiden Indonesia pertama?",
    "Siapa penemu alat musik gitar?",
    "Hitung luas lapangan bola."
]

for q in questions_out_of_context:
    print(f"\n{'-'*5}")
    print(f"Pertanyaan: {q}")
    chatbot_answer = ask(q)
    print(f"Jawaban Chatbot:\n{chatbot_answer}")

    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'-'*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'-'*50}")
    




 UJI COBA 2: Pertanyaan di Luar Konteks 

-----
Pertanyaan: Apa itu Allo Bank?
Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Bagaimana cara mengajukan KPR?
Jawaban Chatbot:
Pertanyaan Peserta: Bagaimana cara mengajukan KPR?

Jawaban Profesional:

Untuk mengajukan Klaim Pekerjaan Residu (KPR), peserta dapat melakukan langkah-langkah berikut:


1.  Peserta harus memastikan bahwa mereka telah memenuhi syarat untuk menerima manfaat JKP.

2.  Pastikan semua data dan dokumen yang diperlukan sudah lengkap dan sesuai dengan persyaratan BPJS Ketenagakerjaan.

3.  Peserta dapat mengajukan klaim KPR melalui Sistem Informasi Ketenagakerjaan (SIK) atau melalui pihak pemberi kerja mereka.

4.  P

Total pertanyaan diluar konteks dataset : 25
- Pertanyaan terjawab:3

- Pertanyaan tidak terjawab:22

In [11]:
# TEST CASE 3: Pertanyaan dalam konteks namun ada kesalahan penulisan
print("\n\n UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan penulisan ")
questions_context_typo = [
    "Aph itu BPJS Ketenagakerajaan?",
    "Apa sajak program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?",
    "Apakh yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?", 
    "Gimana jika lupa PIN EPS?",
    "Apakah itu Perumahan Pekerja?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Kapan BSU tahun 2025 dibagikan?",
    "Dimana obat batuk pertama kali ditemukan",
    "Jika flu sebaiknya minum obat herbal apa",
    "Sopo peserta BPJS ketenagakerjaan",
    "Sejak tanggal berapa pembayaran iuran paling lambat ?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?",
    "Apakah itu ShoopePay?",
    "Apa perbedaan program Jaminan Dari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Gimana pembayaran iuran?",
    "Tanggal pembayaran iuran paling lambat ?",
    "Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?"
    
]

for q in questions_context_typo:
    print(f"\n{'-'*5}")
    print(f"Pertanyaan: {q}")
    chatbot_answer = ask(q)
    print(f"Jawaban Chatbot:\n{chatbot_answer}")

    
    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'-'*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'-'*50}")
    



 UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan penulisan 

-----
Pertanyaan: Aph itu BPJS Ketenagakerajaan?
Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Apa sajak program BPJS Ketenagakerjaan?
Jawaban Chatbot:
Pertanyaan peserta sangat penting! Berikut adalah jawaban profesional:

BPJS Ketenagakerjaan memiliki beberapa program yang ditujukan untuk penerima upah, yaitu:


1. BPJS Ketenagakerjaan Sederhana

2. BPJS Ketenagakerjaan Premium

3. BPJS Ketenagakerjaan Plus

Setiap program ini memiliki kelebihan dan manfaat yang berbeda-beda. Untuk mendapatkan informasi lebih lanjut tentang setiap program, Anda dapat mengunjungi situs web resmi BPJS Ketenagakerjaan a

Total pertanyaan salah tidak baku/typo : 20

- Pertanyaan -jawaban ada di dataset dan terjawab:1
- Pertanyaan -jawaban tidak ada di dataset dan terjawab:11
- Pertanyaan - jawaban ada di dataset dan tidak terjawab:2
- Pertanyaan - tidak ada di dataset dan tidak terjawab:6

## CONCLUSION

Berdasarkan serangkaian pengujian terhadap chatbot yang dibangun menggunakan model LLAMA 3.2 dan berbasis pada dataset FAQ tertentu, dapat disimpulkan bahwa:

1. Kemampuan Memahami Pertanyaan - Chatbot mampu menjawab sebagian besar pertanyaan yang ada di dataset, namun akurasi jawaban masih bervariasi. Beberapa jawaban kurang tepat meskipun informasi tersedia dalam data.
2. Kemampuan Menjaga Batasan Konteks - Model belum sepenuhnya konsisten dalam menjaga batasan topik. Masih terdapat beberapa pertanyaan di luar konteks yang tetap dijawab, menunjukkan perlunya penyempurnaan dalam kontrol domain.

3. Kemampuan Menolak dan Menyampaikan Ketidaktahuan - Model cenderung tetap menjawab meskipun tidak memiliki informasi yang relevan. Bahkan, dalam beberapa kasus memberikan dua jenis jawaban: penjelasan dan pernyataan bahwa model tidak tahu, yang dapat membingungkan pengguna.

4. Kemampuan Mengatasi Pertanyaan dengan Typo - Chatbot cukup tangguh terhadap pertanyaan dengan typo ringan atau bahasa tidak baku, namun akurasi menurun jika kesalahan penulisan mengubah makna.

Model LLAMA 3.2

Kelebihan:

* Mampu memahami pertanyaan dalam konteks, termasuk dengan typo ringan.
* Responsif dan tetap menjawab meski format pertanyaan tidak baku.

Kelemahan:

* Tidak konsisten dalam menolak pertanyaan di luar konteks.
* Beberapa jawaban kurang akurat meski informasi tersedia.
* Memberikan respons ganda (jawaban + pengakuan tidak tahu).

